In [1]:
import json
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA

In [2]:
# 1. 读取 JSON 格式的知识库
json_path = "rag-test-KB.json"
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

In [4]:
# 2. **创建 Document 对象**
documents = []
for item in data:
    knowledge = item["knowledge"]
    doc = Document(page_content=knowledge, metadata={"source": item["knowledge"]})
    documents.append(doc)

In [5]:
# 3. **使用 HuggingFace 本地嵌入模型**
embedder = HuggingFaceEmbeddings(model_name="./all-MiniLM-L6-v2")

C:\Users\11032\AppData\Local\Temp\ipykernel_29988\3068361344.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="./all-MiniLM-L6-v2")


In [6]:
# 4. **创建 FAISS 向量数据库**
vector_store = FAISS.from_documents(documents, embedder)
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3})

In [7]:
# 5. **使用 DeepSeek 1.5B 作为 LLM**
llm = Ollama(
    model="deepseek-r1:1.5b",
    system="你是一个严格的 AI 助手，你只能基于提供的上下文回答问题。\n"
           "如果你无法从上下文中找到答案，请回答 '我不知道'。\n"
           "不要编造信息。\n"
           "你必须引用相关内容，并用 `【来源: source】` 标明出处。"
)

C:\Users\11032\AppData\Local\Temp\ipykernel_29988\3372562694.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [8]:
# 6. **定义 QA Prompt（适用于长知识）**
QA_PROMPT = PromptTemplate.from_template(
    """
    你是一个专业的知识助手，你只能基于提供的上下文回答问题。\n
    请确保你的回答只包含来自上下文的信息，不要编造内容。\n
    你必须引用相关内容，并用 `【来源: source】` 标明出处。\n
    相关知识:
    {context}
    {question}
    答案:
    """
)

In [9]:
# 7. **创建 LLMChain**
llm_chain = LLMChain(llm=llm, prompt=QA_PROMPT, verbose=False)

C:\Users\11032\AppData\Local\Temp\ipykernel_29988\3571466328.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=QA_PROMPT, verbose=False)


In [10]:
# 8. **文档处理链**
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
)

C:\Users\11032\AppData\Local\Temp\ipykernel_29988\2683357691.py:2: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(


In [11]:
# 9. **创建 RetrievalQA**
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
)

C:\Users\11032\AppData\Local\Temp\ipykernel_29988\2122260647.py:2: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  qa = RetrievalQA(


In [12]:
# 10. **LLM 自检 Prompt（防止幻觉）**
CHECK_PROMPT = PromptTemplate.from_template(
    """
    你刚刚生成了以下答案:
    "{answer}"
    请检查这个答案是否严格基于以下上下文:
    {context}
    如果答案和上下文不匹配，请回答："这个答案可能包含幻觉"。
    如果答案完全来自上下文，请回答："这个答案是基于上下文的"。
    """
)
llm_chain_check = LLMChain(llm=llm, prompt=CHECK_PROMPT, verbose=False)

In [16]:
# 11. **用户输入查询**
query = "国足能进世界杯吗"

In [17]:
# 12. **检索相关文档**
search_results = retriever.get_relevant_documents(query)

In [19]:
# 13. **检查相似度，避免误答**
similarity_threshold = 0.5
if not search_results or min([doc.metadata.get("score", 1) for doc in search_results]) < similarity_threshold:
    print("\n回答: 我不知道")
else:
    # 14. **生成回答**
    response = qa(query)
    # 15. **让 LLM 自检是否有幻觉**
    context_text = "\n\n".join([doc.page_content for doc in search_results])
    check_response = llm_chain_check.run({"answer": response["result"], "context": context_text})
    if "幻觉" in check_response:
        print("\n⚠️ 该回答可能包含幻觉，建议用户谨慎使用！")
    else:
        print("\n✅ 该回答是基于上下文的。")
    print("\n最终回答:", response["result"])


✅ 该回答是基于上下文的。

最终回答: <think>
好的，我现在要回答用户的问题“国家足能否进世界杯”。首先，我需要判断是否有相关法规或标准支持这一点。根据之前的对话和知识库内容，我记得有关于无菌药品生产标准中提到的自净时间。不过，关于足球比赛的规则，特别是关于是否可以参赛进入世界杯 tournament，这应该是国际足联的内部政策。

根据我的信息，国际足联规定，球队必须通过资格审查，并且只能代表国家参加世界杯 tournament。因此，如果国家足没有资格入选，就无法进世界杯。这一点是明确的，所以我的回答应该基于这一官方立场。
</think>

国家足不能进入世界杯 tournament，因为国际足联将严格筛选球队资格并Only allow qualifying teams to participate.
